In [3]:
from sqlalchemy import create_engine
import pandas as pd 
import numpy as np
mysql_setting = {
    'host' : "10.10.10.4",
    'dbname': "spider",
    'user': "root",
    'password': "qwerasdf"
}

def getDatabaseConn():
    engine = create_engine('mysql://{user}:{password}@{host}:3306/{dbname}'.format(**mysql_setting), echo=False)
    conn = engine.connect()
    return conn
conn = getDatabaseConn()

/Users/monstar/Envs/py3env/lib/python3.6/site-packages/sqlalchemy/dialects/mysql/base.py:1569: Warning: (1287, "'@@tx_isolation' is deprecated and will be removed in a future release. Please use '@@transaction_isolation' instead")
  cursor.execute('SELECT @@tx_isolation')


In [33]:
def salarySplit(line):
    import re
    res = re.match(r'([\d]+)K-([\d]+)K', line)
    if not res:
        res = re.match(r'([\d]+)k-([\d]+)k', line)
    if res:
        salary_low = res[1]
        salary_high = res[2]
        salary_mean = (int(salary_low) + int(salary_high)) / 2
    else:
        res = re.match(r'([\d]+).*', line)
        salary_low = res[1]
        salary_high = res[1]
        salary_mean = res[1]
    return pd.Series([int(salary_low) * 1000, int(salary_high) * 1000])

In [16]:
zl_res = pd.read_sql("SELECT position_name, salary_low, salary_high, work_year, education, recruit_num, position_type FROM spider.zhilian_position where city = '成都'", conn)
sql = """
    select publish_time, position_name, position_labels, salary, work_year, education, second_type, city, department
    from lagou_recruit_day where city = '成都'
"""
lg_res = pd.read_sql(sql, conn)

In [35]:
salary_df = lg_res['salary'].apply(salarySplit).rename(columns={0:'salary_low', 1:'salary_high'})
lg_res_deal = pd.concat([lg_res, salary_df] ,axis=1)
res = pd.concat([lg_res_deal[['position_name', 'work_year', 'education', 'salary_low', 'salary_high']], zl_res[['position_name', 'work_year', 'education', 'salary_low', 'salary_high']]])

In [42]:
res[(res.position_name.str.contains('u3d')) | (res.position_name.str.contains('unity'))].to_excel('/Users/monstar/Desktop/unity3D职位情况.xlsx')

In [47]:
res[(res.position_name.str.contains('u3d')) | (res.position_name.str.contains('unity'))].pivot_table(index='work_year',columns=['education']).fillna('--').rename(columns={'salary_high':'薪资上线', 'salary_low': '薪资下限'})

薪资上线                          薪资下限                     
education       不限            大专     本科       不限            大专     本科
work_year                                                            
1-3年          9375  12285.714286   8000  6375.62   7786.142857   6000
1年以下            --   8666.666667     --       --   4667.000000     --
3-5年       20333.3  17400.000000  23000    14000  11600.400000  14000
不限           15800   4250.000000   8000   9200.4   2500.750000   4667